<center> <h1> Feature Engineering and data cleansing with RAPIDS </h1> </center>

In [ ]:
import cuml
import cudf
import nvcategory

import xgboost as xgb
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error, roc_auc_score

#### Read in the data to GPU memory. 

In [ ]:
#If you put the zip file in the root folder, notice how it decompresses as it reads the data into memory.
gdf = cudf.read_csv('train.csv')

In [ ]:
#Taking a look at the data. We use "to_pandas()" to get the pretty printing. 
gdf.head().to_pandas()

#### EXERCISE: Let's do some descriptive statistics 

In [ ]:
#Hint: try some of the function you may know from Pandas like DataFrame.Series.max() or look up the documentation here:

In [ ]:
#grabbing the first character of the years in city string to get rid of plus sign, and converting to int
<Your Code Goes Here>

In [ ]:
#Here we can see how we can control what the value of our dummies with the replace method and turn strings to ints
gdf['City_Category'] = gdf.City_Category.str.replace('A', '1')
gdf['City_Category'] = gdf.City_Category.str.replace('B', '2')
gdf['City_Category'] = gdf.City_Category.str.replace('C', '3')
gdf['City_Category'] = gdf['City_Category'].str.stoi()

#### EXERCISE: replace city in the same way as City Category

In [ ]:
#Hint: the Gender column only has values 'M' and 'F'

In [ ]:
#Solution
<Your Code Goes Here>

In [ ]:
#Let's take a look at how many products we have
prod_count = cudf.Series(nvcategory.from_strings(gdf.Product_ID.data).values()).unique().count() #hideous one-liner
print("Unique Products: {}".format(prod_count))

In [ ]:
#Let's take a look at how many primary product categories we have
#We do it differently here because the variable is a number, not a string
prod1_count = gdf.Product_Category_1.unique().count()
print("Unique Product Categories: {}".format(prod1_count))

In [ ]:
#Filling missing values
gdf['Product_Category_2'] = gdf['Product_Category_2'].fillna(0)

In [ ]:
#EXERCISE: Make a variable that's 1 if the product is multi-category, 0 otherwise

In [ ]:
#Hint: think about how to combine the Product Category 2 and Product Category 3

In [ ]:
#Solution: 
<Your Code Goes Here>

In [ ]:
#EXERCISE: Create a Gender/Marital Status Interaction Effect

In [ ]:
#Hint: bother Gender and Marital Status are 0/1

In [ ]:
#Solution:
<Your Code Goes Here>

In [ ]:
#Because Occupation is a code, it should converted into indicator variables
gdf = gdf.one_hot_encoding('Occupation', 'occ_dummy', gdf.Occupation.unique())

In [ ]:
#Dummy variable from Int
gdf = gdf.one_hot_encoding('City_Category', 'city_cat', gdf.City_Category.unique())

#Dummy from string
cat = nvcategory.from_strings(gdf.Age.data)
gdf['Age'] = cudf.Series(cat.values())
gdf = gdf.one_hot_encoding('Age', 'age', gdf.Age.unique())

#### EXERCISE: Create dummy variables from Product Category 1

In [ ]:
#Solution:
<Your Code Goes Here>

In [ ]:
#We're going to drop th variables we've transformed
drop_list = ['User_ID', 'Age', 'Stay_In_Current_City_Years', 'City_Category','Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']
gdf = gdf.drop(drop_list)

In [ ]:
#We're going to make a list of all the first indicator variables in a series now so it will be
#easier to exclude them when we're doing regressions later

In [ ]:
dummy_list = ['occ_dummy_0', 'city_cat_1', 'age_0', 'product_1', 'Purchase']

In [ ]:
#All variables currently have to have the same type for some methods in cuML
for col in gdf.columns.tolist():
    gdf[col] = gdf[col].astype('float32')

In [ ]:
test_size = round(len(gdf)*0.2)
train_size = round(len(gdf)-test_size)

In [ ]:
test = gdf.iloc[0:test_size]

#### EXERCISE: Make the test set in a similar way

In [ ]:
#Solution:
<Your Code Goes Here>

In [ ]:
#Deleting the main gdf because we're going to be making other subsets and other stuff, so it will be nice to have the memory. 
del(gdf)

In [ ]:
y_train = gdf_train['Purchase']
X_reg = gdf_train.drop(dummy_list)

In [ ]:
# # I'm going to perform a hyperparameter search for alpha in a ridge regression
output = {}
for alpha in np.around(np.arange(0.1, 10, 0.1), decimals=2):
    
    Ridge = cuml.Ridge(alpha=alpha, fit_intercept=True)
    _fit = Ridge.fit(X_reg, y_train)
    _y_hat = _fit.predict(X_reg)
    _mse = sqrt((y_train.reset_index(drop=True).sub(_y_hat)**2).sum())
    output['MSE_RIDGE_{}'.format(alpha)] = _mse

print('MAX AUC: {}'.format(min(output, key=output.get)))

In [ ]:
Ridge = cuml.Ridge(alpha=.1, fit_intercept=True)
_fit = Ridge.fit(X_reg, y_train)
_y_hat = _fit.predict(X_reg)
_mse = sqrt((y_train.reset_index(drop=True).sub(_y_hat)**2).sum())
print('{:,}'.format(_mse))

In [ ]:
y_train.max()

In [ ]:
y_xgb = gdf_train[['Purchase']]
X_xgb = gdf_train.drop('Purchase')
xgb_train_set = xgb.DMatrix(data=X_xgb, label=y_xgb)

In [ ]:
xgb_params = {
    'nround':100,
    'max_depth':4,
    'max_leaves':2**4,
    'tree_method':'gpu_hist',
    'n_gpus':1,
    'loss':'ls',
    'objective':'reg:squarederror',
    'max_features':'auto',
    'criterion':'friedman_mse',
    'grow_policy':'lossguide',
    'verbose':True
}

In [ ]:
xgb_model = xgb.train(xgb_params, dtrain=xgb_train_set)

In [ ]:
y_hat_xgb = xgb_model.predict(xgb_train_set)

In [ ]:
RMSE = np.sqrt(mean_squared_error(y_xgb['Purchase'].to_pandas(), y_hat_xgb)) #get out of sample RMSE too

In [ ]:
print(RMSE)

#### EXERCISE: Change XGB around to predict if someone is married based on the data we have. Also print the AUC

In [ ]:
#Hint: in the xgb parameters, change the objective function to 'reg:logistic'

In [ ]:
#Solution
<Your Code Goes Here>

#### OPTIONAL EXERCISE

In [ ]:
#EXTRA EXERCISE: Apply kNN to the customers
#EXTRA EXERCISE: Apply PCA to data